<a href="https://colab.research.google.com/github/Goro-python/my_NLP/blob/master/SeqtoSeq_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
# coding "a+b = c"through seq to seq modelling 

In [110]:
# generate the data 
import numpy as np 
np.random.seed(1) 
def genTrainingData(maxDigit,sizeData): 
  X = []
  Y = []
  digits= list("123456789")
  maxLen = maxDigit+1 
  for count in range(sizeData):
    a = ""
    b = ""
    c = None 
    for i in range(maxDigit):
      a+=np.random.choice(digits)
      b+=np.random.choice(digits)
    exp = a+"+"+b
    #print(a)
    #print(b)
    c = str(int(a) + int(b))
    #print(c)
    X.append(exp[::-1])
    Y.append((" "*(maxLen-len(c))+c)[::-1])
  return [X,Y] 

In [111]:
# testcase
Xtrain,Ytrain = genTrainingData(3,50000)
XVal, YVal =    genTrainingData(3,4000)
Xtest,Ytest =   genTrainingData(3,2000)

In [112]:
# create the conversion data structure
Xtrain[:10]

['219+166',
 '357+638',
 '283+855',
 '281+778',
 '992+491',
 '464+278',
 '459+125',
 '831+853',
 '587+849',
 '394+176']

In [113]:
Ytrain[0:10]

['3751',
 '9851',
 '049 ',
 '9501',
 '394 ',
 '6331',
 '5741',
 '694 ',
 '3371',
 '4611']

In [114]:
from collections import defaultdict
class conversionTable(object):
  def __init__(self,maxLen):
    self.digits = list("0123456789+ ")
    self.indextoChar = defaultdict(str)
    self.chartoIndex = defaultdict(int)
    self.maxLen = maxLen 
  def convertChartoIndex(self):
    count = 0 
    for ch in self.digits:
      self.chartoIndex[ch] = count
      count+=1
  def convertIndextoChar(self):
    for key,val in self.chartoIndex.items():
      self.indextoChar[val] = key

  def convertExptoArr(self,exp):
    X = np.zeros((len(exp), len(self.digits)))
    for i,ch in enumerate(exp):
      index = self.chartoIndex[ch]
      X[i,index] = 1
    return X

  def convertArrtoExp(self,Arr):  # it is going to convert the 2D array in to an expression using argmax
    exp = ""
    maxArr = np.argmax(Arr,axis = 2)
    maxArr=maxArr.flatten()
    #print(maxArr)
    for index in maxArr:      
      exp+=self.indextoChar[index]
    return exp[::-1]


# tetscase:
ct = conversionTable(4)
ct.convertChartoIndex();
ct.convertIndextoChar();

arr = ct.convertExptoArr("732+321")   
  

In [115]:
arr

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [116]:
# convert training data to number
def convertDatatoNumeric(ct,trainingData):
  X = []
  for data in trainingData:
    convert = list(ct.convertExptoArr(data))
    X.append(convert)
  return np.array(X)

Xtr = convertDatatoNumeric(ct,Xtrain)
Ytr = convertDatatoNumeric(ct,Ytrain)

Xv = convertDatatoNumeric(ct,XVal)
Yv = convertDatatoNumeric(ct,YVal)







In [117]:
print(Ytr[0:10])

[[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]

 [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 1. 0. 0. 0

In [118]:
print(Xtrain[0],Ytrain[0])

219+166 3751


In [119]:
print(Ytr[0])

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [120]:
# build the model
# Deep neural Nets # LSTM mode

from tensorflow import keras
from tensorflow.keras import layers

num_layers = 1  # Try to add more LSTM layers!
MaxLen = 7

model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(layers.LSTM(128, input_shape=(MaxLen, len(ct.digits))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(4))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(num_layers):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(ct.digits), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()





Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense_2 (Dense)              (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [121]:
 # training 
batch_size = 32
# Train the model each generation and show predictions against the validation
# dataset.
shouldITrain = False
if shouldITrain :
  model.fit(Xtr,Ytr,batch_size=batch_size,epochs=5,validation_data = (Xv,Yv))

In [122]:
Xtst = convertDatatoNumeric(ct,Xtest)
Ytst = convertDatatoNumeric(ct,Ytest)

In [123]:
# checking Accuracy 
from sklearn.metrics import accuracy_score
y_pred = model.predict(Xtst)


In [124]:
np.argmax(y_pred[0],axis=1)

array([0, 0, 0, 0])

In [125]:
y_pred[0]

array([[0.08436313, 0.08365491, 0.08295435, 0.08326829, 0.0825467 ,
        0.08431403, 0.08311092, 0.08367511, 0.08346473, 0.08285223,
        0.0834671 , 0.0823285 ],
       [0.08521505, 0.08387945, 0.08273798, 0.08326889, 0.08190183,
        0.08494324, 0.08291934, 0.0839176 , 0.08360329, 0.08245141,
        0.08363663, 0.0815253 ],
       [0.08591323, 0.08403631, 0.08262551, 0.08330656, 0.08137444,
        0.08532463, 0.08275345, 0.0841009 , 0.08374534, 0.08213378,
        0.08381125, 0.08087464],
       [0.08648118, 0.08414627, 0.08257827, 0.08336391, 0.08094446,
        0.08553556, 0.08260872, 0.08424631, 0.08388598, 0.08189296,
        0.08397336, 0.080343  ]], dtype=float32)

In [126]:
shouldITrain = True
if shouldITrain :
  model.fit(Xtr,Ytr,batch_size=batch_size,epochs=10,validation_data = (Xtst,Ytst))

Epoch 1/10
1563/1563 [==============================] - 24s 15ms/step - loss: 1.4646 - accuracy: 0.4477 - val_loss: 0.7622 - val_accuracy: 0.7299
Epoch 2/10
1563/1563 [==============================] - 24s 15ms/step - loss: 0.3057 - accuracy: 0.9330 - val_loss: 0.0894 - val_accuracy: 0.9939
Epoch 3/10
1563/1563 [==============================] - 23s 15ms/step - loss: 0.0627 - accuracy: 0.9925 - val_loss: 0.0242 - val_accuracy: 0.9991
Epoch 4/10
1563/1563 [==============================] - 23s 15ms/step - loss: 0.0346 - accuracy: 0.9932 - val_loss: 0.0087 - val_accuracy: 1.0000
Epoch 5/10
1563/1563 [==============================] - 23s 15ms/step - loss: 0.0276 - accuracy: 0.9935 - val_loss: 0.2453 - val_accuracy: 0.9149
Epoch 6/10
1563/1563 [==============================] - 23s 15ms/step - loss: 0.0115 - accuracy: 0.9975 - val_loss: 0.0050 - val_accuracy: 0.9995
Epoch 7/10
1563/1563 [==============================] - 23s 15ms/step - loss: 0.0255 - accuracy: 0.9938 - val_loss: 0.0023 -

In [127]:
# coverting 
#Xtst[0:1].shape


In [128]:
testSize = 2000
for i in range(5):
  index = np.random.randint(0,testSize)
  Y = model.predict(Xtst[index:index+1])
  res = ct.convertArrtoExp(Y)
  print("Input: ", Xtest[index][::-1])
  print("True Output: ",Ytest[index][::-1])
  print("ML output",res)


Input:  398+772
True Output:  1170
ML output 1170
Input:  264+225
True Output:   489
ML output  489
Input:  615+212
True Output:   827
ML output  827
Input:  278+914
True Output:  1192
ML output 1192
Input:  178+861
True Output:  1039
ML output 1039


In [144]:
a = input("Enter first digit number : ")
b = input("Enter second digit number")
print(a,b)

Enter first digit number : 455
Enter second digit number567
455 567


In [145]:
addExp = str(a)+"+"+str(b)
addExp = addExp[::-1]
X = ct.convertExptoArr(addExp)

In [146]:
X = np.reshape(X,(1,7,12))

In [147]:
X.shape

(1, 7, 12)

In [148]:
Y = model.predict(X)

In [149]:
Y.shape

(1, 4, 12)

In [150]:
res = ct.convertArrtoExp(Y)

In [151]:
res

'1022'